# 単純ベイズの実装

## 概論

特徴がカテゴリ変数である場合の単純ベイズ法による分類

変数を次のように定義する。

・特徴量：$\mathbf{x_i}=(x_{i1,x_{i2},\cdots,x_{iK}})$...カテゴリ変数であり、$F_j$個の値のうちの一つを取る。$K$は特徴の種類数である。

・クラス：$y$...$C$個の値のうち一つを取る

$\mathbf{X}$と$Y$の同時分布は次式で与えられる。

\begin{equation}
    P(\mathbf{X}, Y) = P(Y) \prod_{j=1}^K P(X_j|Y)
\end{equation}

$P(Y)$と$P(X_j|Y)$の分布が離散分布である場合、
学習すべき単純ベイズのパラメータは、$P(y)$と$P(X_j|y)$の2つである。
$(y=0,1,\cdots,C, \,\, x_j=1,2,\cdots,F_j, \,\,j=1,2,\cdots,K)$

さらに、$C$や$F_j$は全て1に固定すると、クラスや各特徴量は全て2値変数となり、
$P(y)$と$P(X_j|y)$はベルヌーイ分布に従う。

大きさ$N$のデータ集合$\mathcal{D}=\{ \mathbf{x_i},y_i \}$、$i=1,2,\cdots,N$が与えられると、
対数尤度は次式で表される。

\begin{equation}
    L\left(\mathcal{D};{P(y)},{P(x_j|y)}\right) = \sum_{\{\mathbf{x_i}, y_i\} \in \mathcal{D}} \ln P(\mathbf{x_i}, y_i)
\end{equation}

この対数尤度を最大化する最尤推定を行い、$P(y)$と$P(X_j|y)$を求める。

このうち、クラスの分布のパラメータ群$P(y)$は次式で計算される。

\begin{equation}
    P(y)=\frac{N[y_i=y]}{N}, y \in \{0,1\}
\end{equation}

ただし、$N[y_i=y]$は、データ集合$\mathcal{D}$のうち、クラス$y_i$の値が$y$である数である。

もう一つのパラメータ群$P(X_j|y)$は次式となる。

\begin{equation}
    P(X_j|y)=\frac{N[x_{ij}=x_j,y_i=y]}{N[y_i=y]}
\end{equation}

ただし、$N[x_{ij}=x_j,y_i=y]$は、データ集合$\mathcal{D}$のうち、
クラス$y_i$の値が$y$であり、かつ特徴$x_{xy}$の値が$x_{j}$である事例の数である。

$(y \in {0,1}, \,\, x_i \in {0,1}, \,\, j=1,2,\cdots,K)$

以上のパラメータの計算に必要な$P(x_{ij}=x{ij}|y_{j}=y)$および$N[x_{ij}=x_{j},y_{i}=y]$は、
データ集合$\mathcal{D}$に対して数え上げることにより計算可能である。

予測を行う際には、入力ベクトル$\mathbf{x}^{\text{new}}$が与えられたときのクラス事後確率を
最大にするクラスを次式で求める。

\begin{equation}
    \begin{aligned}
        \hat{y} &=\arg \max _{y} P\left(y | \mathbf{x}^{\text {new }}\right) \\
        &=\arg \max _{y} \frac{P(y) P\left(\mathbf{x}^{\text {new }} | y\right)}{\sum_{y^{\prime}} P\left(y^{\prime}\right) P\left(\mathbf{x}^{\text {new }} | y^{\prime}\right)} \\
        &=\arg \max _{y} P(y) P\left(\mathbf{x}^{\text {new }} | y\right) \\
        &=\arg \max _{y}\left(P(y) \prod_{j} P\left(x_{j}^{\text {new }} | y\right)\right) \\
        &=\arg \max _{y}\left(\log P(y)+\sum_{j} \log P\left(x_{j}^{\text {new }} | y\right)\right)
    \end{aligned}
\end{equation}

この式は、求めたパラメータ$P(y)$、$P(X_j|y)$を利用して計算される。

なお、最後に対数を取っているのは、浮動小数点計算による計算結果の不安定さを回避するためである。


## NaiveBayesクラスの実装

In [1]:
import numpy as np


class NaiveBayes():
    
    def __init__(self):
        self.pY_ = None    # p(y)
        self.pXgY_ = None  # p(xi | y)

    def fit(self, X, y):
        
        # 定数を定義
        n_samples = X.shape[0]   # サンプル数
        n_features = X.shape[1]  # 特徴数
        n_classes = 2  # クラス数 (2値)
        n_fvalues = 2  # 特徴数 (2値)

        # 特徴の事例数とクラスラベルの事例数は一致していなければならない
        if n_samples != len(y):
            raise ValueError('Mismatched number of samples.')

        # N[yi=y] の数を数え上げる
        nY = np.zeros(n_classes, dtype=int)
        for i in range(n_samples):
            nY[y[i]] += 1

        # pY_ (p(y_i)) を計算する
        self.pY_ = np.empty(n_classes, dtype=float)
        for i in range(n_classes):
            self.pY_[i] = nY[i] / n_samples

        # N[x_ij=xj, yi=y] の数を数え上げる
        nXY = np.zeros((n_features, n_fvalues, n_classes), dtype=int)
        for i in range(n_samples):
            for j in range(n_features):
                nXY[j, X[i, j], y[i]] += 1

        # pXgY_ (p(xi | y)) を計算する
        self.pXgY_ = np.empty((n_features, n_fvalues, n_classes),dtype=float)
        for j in range(n_features):
            for xi in range(n_fvalues):
                for yi in range(n_classes):
                    self.pXgY_[j, xi, yi] = nXY[j, xi, yi] / nY[yi]

    def predict(self, X):

        n_samples = X.shape[0]
        n_features = X.shape[1]
        y = np.empty(n_samples, dtype=int)

        # 対数同時確率を計算
        for i, xi in enumerate(X):
            logpXY = (np.log(self.pY_) +
                      np.sum(np.log(self.pXgY_[np.arange(n_features), xi, :]), axis=0))
            # クラスを予測
            y[i] = np.argmax(logpXY)
            
        return y

## 予測の実行
データセットにはUCI Repositpryの[Congressional Voting Records Data Set](http://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records)を用いる。
これはアメリカ議会での16種の議題に対する投票行動を特徴とし，議員が共和党 (0) と民主党 (1) のいずれであるかがクラスである。

In [2]:
data_dir = './../data/'
data = np.genfromtxt(data_dir+'vote_filled.tsv', dtype=int)
X = data[:, :-1]
y = data[:, -1]

bayes=NaiveBayes()
bayes.fit(X,y)

predict_y = bayes.predict(X[:10, :])
print('n T P')
for i in range(10):
    print(i, y[i], predict_y[i])

n T P
0 1 1
1 1 1
2 0 0
3 0 0
4 0 0
5 0 0
6 0 1
7 1 1
8 1 1
9 0 0


## 参考文献
Toshihiro Kamishima "Machine Learning Meets Python" https://github.com/tkamishima/mlmpy online accessed:2020/05/17